<a href="https://colab.research.google.com/github/benmsanderson/tcre-star/blob/main/ESGF_read_1pct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp /content/drive/MyDrive/esmf/esmf.tar /content

In [4]:
os.chdir('/content')
!tar -xf esmf.tar

In [5]:
!python3 -m pip install /content/esmf/src/addon/esmpy

Processing ./esmf/src/addon/esmpy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for esmpy: filename=esmpy-8.6.0b7-py3-none-any.whl size=120542 sha256=7bc1c17e238a8d8e87377ddcd9b05fffce3d7faaf5ae022435b4b0de59f6ed6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-8y0_inby/wheels/32/fe/f1/f7ade03cece49b78d0fe0ad7db65bca3927bb369ac4eb124bd
Successfully built esmpy
  Attempting uninstall: esmpy
    Found existing installation: esmpy 8.6.0b7
    Uninstalling esmpy-8.6.0b7:
      Successfully uninstalled esmpy-8.6.0b7


In [6]:
pip install git+https://github.com/pangeo-data/xesmf.git

  Cloning https://github.com/pangeo-data/xesmf.git to /tmp/pip-req-build-afp0_guq
  Running command git clone --filter=blob:none --quiet https://github.com/pangeo-data/xesmf.git /tmp/pip-req-build-afp0_guq
  Resolved https://github.com/pangeo-data/xesmf.git to commit 35a872743d93205239bd00e84e54e85f5c393f0e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
os.environ["ESMFMKFILE"] = '/content/esmf/lib/libO/Linux.gfortran.64.mpiuni.default/esmf.mk'

In [8]:
import xesmf as xe


/usr/local/lib/python3.10/dist-packages/esmpy/interface/loadESMF.py:92: VersionWarning: ESMF installation version 8.6.0 beta snapshot, ESMPy version 8.6.0b7
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [9]:
import warnings
warnings.simplefilter("ignore")

from itertools import groupby

In [10]:
pip install nc-time-axis


In [11]:
pip install h5pyd

In [12]:
 pip install netcdf4==1.6.4

In [13]:
import requests

In [14]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import h5pyd
import cftime
import xarray as xr
import nc_time_axis

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Search using ESGF API

In [15]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, use_replicas=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    payload["latest"]="true"
    if use_replicas:
        payload["replica"]="true"

    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = []
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        #print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    result=sorted(all_files)
    tmp = [[c for _, c in g] for _, g in groupby(zip( [x.split('_')[-3]+'_'+x.split('_')[-2]+'_'+x.split('_')[0].split('http://')[-1].split('/')[0] for x in result],result), key=lambda x: x[0])]
    return tmp

In [16]:
mdls=["CanESM5","GFDL-ESM4","GISS-E2-1-G","GISS-E2-1-H","CESM2-WACCM","MIROC-ES2L","UKESM1-0-LL","IPSL-CM6A-LR"]
#mdls=["CanESM5"]

In [17]:
scens={'piControl','1pctCO2'}
vars={'fgco2','nbp','tas'}
var_types={}
var_types['nbp']='Lmon'
var_types['tas']='Amon'
var_types['fgco2']='Omon'



In [18]:
def pullfile(fl):
  lst=[]
  for f in fl:
    url = f.replace('dodsC','fileServer')
    fn=url.split('/')[-1]
    r = requests.get(url, allow_redirects=True)
    if os.path.isfile(fn):
      print('exists')
    else:
      open(fn, 'wb').write(r.content)
    lst.append(fn)
  return lst

In [19]:
result_scn={}

for i,mdl in enumerate(mdls):
  result_scn[mdl]={}
  for j,var in enumerate(vars):
        result_scn[mdl][var]={}
        for k,scn in enumerate(scens):
              result_scn[mdl][var][scn] = esgf_search(activity_id='CMIP', table_id=var_types[var],
                            experiment_id=scn, variable_id=var,
                            source_id=mdl)
              if len(result_scn[mdl][var][scn])>0:
                response = requests.get(result_scn[mdl][var][scn][0][0])
                if response.status_code != 200:
                  result_scn[mdl][var][scn] = esgf_search(activity_id='CMIP', table_id=var_types[var],
                            experiment_id=scn, variable_id=var,
                            source_id=mdl,use_replicas=True)


In [20]:
result_area={}
for i,mdl in enumerate(mdls):

  result_area[mdl] = esgf_search(variable_id='areacella', activity_id='CMIP',
                          source_id=mdl)

In [26]:
lst[:]

['fgco2_Omon_CanESM5_piControl_r1i1p1f1_gn_520101-540012.nc',
 'fgco2_Omon_CanESM5_piControl_r1i1p1f1_gn_540101-560012.nc',
 'fgco2_Omon_CanESM5_piControl_r1i1p1f1_gn_560101-580012.nc',
 'fgco2_Omon_CanESM5_piControl_r1i1p1f1_gn_580101-600012.nc',
 'fgco2_Omon_CanESM5_piControl_r1i1p1f1_gn_600101-620012.nc']

In [28]:
ds_scn={}
for i,mdl in enumerate(mdls):
    print(mdl)
    ds_scn[mdl]={}
    for j,scn in enumerate(scens):
      print(scn)
      tmp=[]
      tmp.append(xr.open_dataset(result_area[mdl][0][0]))
      for k,var in enumerate(vars):
       #print(var)
       if len(result_scn[mdl][var][scn])>0:

              fn=result_scn[mdl][var][scn][0]
              grd=fn[0].split('_')[-2]
              if var=='fgco2':
                 lst=pullfile(fn)
                 tmps = xr.open_mfdataset(lst, concat_dim="time",combine="nested",use_cftime=True,coords='minimal')
              else:
                 tmps = xr.open_mfdataset(fn, concat_dim="time",combine="nested",use_cftime=True,coords='minimal')

              if var=='fgco2':
                regridder = xe.Regridder(tmps, tmp[0]['areacella'], "bilinear",ignore_degenerate=True)
                tmps=regridder(tmps)
              tmpc=tmps[var]
  #        if var=='fgco2':
  #            regridder = xe.Regridder(tmpc, tmp[0]['areacella'], "bilinear",ignore_degenerate=True)
  #            tmp[0]['fgco2']=regridder(tmpc)
  #        else:
              tmp.append(tmpc)
      ds_scn[mdl][scn]=xr.merge(tmp,compat='override')


CanESM5
piControl
exists
exists
exists
exists
exists
1pctCO2
GFDL-ESM4
piControl
1pctCO2
GISS-E2-1-G
piControl
1pctCO2
GISS-E2-1-H
piControl
1pctCO2
CESM2-WACCM
piControl
1pctCO2
MIROC-ES2L
piControl
1pctCO2
UKESM1-0-LL
piControl
1pctCO2
IPSL-CM6A-LR
piControl
1pctCO2


In [29]:
datadir='/content/drive/MyDrive/tcre_star'

In [ ]:

for i,mdl in enumerate(mdls):
  print(mdl)
  for j,scn in enumerate(scens):
    print(scn)
    lst=[]
    for v,var in enumerate(vars):
      if len(result_scn[mdl][var][scn])>0:
        print(var)
        tmp=ds_scn[mdl][scn][var].isel(time=slice(None,12*150)).load()
        #tmp=xr.concat([tmp1,tmp2,tmp3,tmp4,tmp5],dim='time')
        #tmp=ds_scn[mdl][scn][var].load()
        tmp=tmp.groupby('time.year').mean()
        lst.append(tmp)

    ot=xr.merge(lst)
    ot.to_netcdf("/content/drive/MyDrive/tcre_star/"+mdl+'_'+scn+'.nc')
    ds_scn[mdl][scn].close()

CanESM5
piControl
fgco2
nbp
tas
1pctCO2
fgco2
nbp
tas
GFDL-ESM4
piControl
fgco2
nbp
tas
1pctCO2
fgco2
nbp
tas
GISS-E2-1-G
piControl
fgco2
nbp
tas
1pctCO2
fgco2
tas
GISS-E2-1-H
piControl
tas
1pctCO2
tas
CESM2-WACCM
piControl
fgco2
nbp
tas
1pctCO2
fgco2
nbp
tas
MIROC-ES2L
piControl
fgco2
nbp
tas


In [ ]:
ot.fgco2[0,:,:].plot()

In [ ]:
import csv

with open(datadir+'/mdls.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(mdls)

In [ ]:
with open(datadir+'/scens.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(scens)